In [1]:
import os
from dotenv import load_dotenv
load_dotenv('azure.env')

True

In [2]:
from IPython import get_ipython
from typing_extensions import Annotated

import autogen
from autogen.cache import Cache

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"tags": ["tool"]},  # comment out to get all
)
import autogen  # noqa: E402

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["gpt-4o"]},
    ),
    "temperature": 0,
}

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["gpt-4o-mini"]},
    ),
    "temperature": 0,
}

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

# one way of registering functions is to use the register_for_llm and register_for_execution decorators
@user_proxy.register_for_execution()
@chatbot.register_for_llm(name="python", description="run cell in ipython and return the execution result.")
def exec_python(cell: Annotated[str, "Valid Python cell to execute."]) -> str:
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log

# another way of registering functions is to use the register_function
def exec_sh(script: Annotated[str, "Valid Python cell to execute."]) -> str:
    return user_proxy.execute_code_blocks([("sh", script)])

autogen.agentchat.register_function(
    exec_python,
    caller=chatbot,
    executor=user_proxy,
    name="sh",
    description="run a shell script and return the execution result.",
)

In [5]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent  # noqa: E402

web_surfer = WebSurferAgent(
    "web_surfer",
    llm_config=llm_config,
    summarizer_llm_config=summarizer_llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": os.environ['BING_SUBSCRIPTION_KEY']},
)

In [ ]:
task1 = """
Search the web for information about Microsoft AutoGen
"""

user_proxy.initiate_chat(web_surfer, message=task1)

In [3]:
import requests
from bs4 import BeautifulSoup
import json

def scrape(url: str):
    # scrape website, and also will summarize the content based on objective if the content is too large
    # objective is the original objective & task that user give to the agent, url is the url of the website to be scraped

    print("Scraping website...")
    # Define the headers for the request
    headers = {
        'Cache-Control': 'no-cache',
        'Content-Type': 'application/json',
    }

    # Define the data to be sent in the request
    data = {
        "url": url
    }

    # Convert Python object to JSON string
    data_json = json.dumps(data)

    # Send the POST request
    response = requests.post(
        "https://chrome.browserless.io/content?token=043abdc1-b765-4298-b7b8-b2c39f6ce27e", headers=headers, data=data_json)

    # Check the response status code
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        text = soup.get_text()
        print("CONTENTTTTTT:", text)
        return text
    else:
        print(f"HTTP request failed with status code {response.status_code}")


In [5]:
import autogen
from autogen.agentchat.contrib.web_surfer import WebSurferAgent  # noqa: E402


# Create the assistant agents
content_analyzer = WebSurferAgent(
    name="Content_Analyzer",
    llm_config=llm_config,
    summarizer_llm_config=summarizer_llm_config,
    system_message="You are an expert content analyzer. Your task is to thoroughly analyze the provided blog post or website content and extract key themes, tone, and style."
)

ad_info_gatherer = WebSurferAgent(
    "Ad_Info_Gatherer",
    llm_config=llm_config,
    summarizer_llm_config=summarizer_llm_config,
    system_message="You are an expert at gathering information about sponsored ads. Your task is to navigate to the provided sponsored ad link and extract relevant information for ad creation.",
    browser_config={"viewport_size": 4096, "bing_api_key": os.environ['BING_SUBSCRIPTION_KEY']},
)

ad_creator = autogen.AssistantAgent(
    name="Ad_Creator",
    llm_config=llm_config,
    system_message="""You are an expert ads creator specializing in native advertising. Your task is to craft a narrative for the ad that feels like a natural extension of the blog's content.
Guidelines for ad creation:
 - Match the tone and style of the blog
 - Ensure the ad is contextually relevant and creative, blending seamlessly with the surrounding content.
 - Limit the ad to a maximum of two paragraphs.
 - If images are provided, incorporate them appropriately.
 - Use language and themes that resonate with the blog's audience
 - Be subtle in your approach – the ad should not feel intrusive or overly promotional"""
)


# Create the user proxy agent
user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config=False,
    llm_config=llm_config,
    system_message="""You are a user proxy coordinating the native ad creation process. You will:
1. Provide the blog content link and sponsored ad link to the Content_Analyzer and Ad_Info_Gatherer.
2. Pass their analyses to the Ad_Creator.
3. Return the final formatted Markdown ad. No need to validate the format of markdown"""
)


# Create a group chat
groupchat = autogen.GroupChat(
    agents=[user_proxy,content_analyzer, ad_info_gatherer, ad_creator],
    messages=[],
    max_round=10
)

manager = autogen.GroupChatManager(groupchat=groupchat)

# Start the conversation
result = user_proxy.initiate_chat(
    manager,
    message="""Create a native ad for the following blog post: https://trends.glance.com/newz/space/IN/en/feedpost-now/f-cc428ee2a0-cf888ef9-bafb-5676-b323-86be059fc469
    The sponsored ad link is: https://www.primevideo.com/detail/Chandu-Champion/0MARCAWG5VJK4GIDNLXR1CR0ON
    
    Please gather the necessary information and create a seamless native ad."""
)
# The conversation will continue automatically through the agents until the task is complete

User_Proxy (to chat_manager):

Create a native ad for the following blog post: https://trends.glance.com/newz/space/IN/en/feedpost-now/f-cc428ee2a0-cf888ef9-bafb-5676-b323-86be059fc469
    The sponsored ad link is: https://www.primevideo.com/detail/Chandu-Champion/0MARCAWG5VJK4GIDNLXR1CR0ON
    
    Please gather the necessary information and create a seamless native ad.

--------------------------------------------------------------------------------

Next speaker: Content_Analyzer


>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION visit_page...
Content_Analyzer (to chat_manager):

Address: https://trends.glance.com/newz/space/IN/en/feedpost-now/f-cc428ee2a0-cf888ef9-bafb-5676-b323-86be059fc469
Title: Countdown to Inspiration: Top 10 Space Movies of All Time!
Viewport position: Showing page 1 of 1.
Countdown to Inspiration: Top 10 Space Movies of All Time!

   ![banner](https://g-mob.glance-cdn.com/public/fc/images/f-cc428ee2a0-cf888ef9-bafb-5676-b323-86be059fc469-b.webp) Fee

In [8]:
from IPython.display import display, Markdown, Latex
display(Markdown(result.summary))

### Countdown to Inspiration: Top 10 Space Movies of All Time!

As you immerse yourself in the wonders of space through our top 10 inspiring space movies, why not continue your journey of inspiration with a story closer to home? "Chandu Champion" on Prime Video is an unbelievable tale of resilience and triumph. This film chronicles the life of a man who, against all odds, secured India's first individual gold medal in the Olympics. Just like the space heroes who venture into the unknown, Chandu's unwavering spirit and determination will captivate and inspire you.

Experience the same thrill and emotional depth that you love in space movies with "Chandu Champion." Available now on Prime Video, this film promises to take you on an extraordinary journey of human endurance and victory. Watch it today and let Chandu's story fuel your own dreams and aspirations. [Watch Now](https://www.primevideo.com/detail/Chandu-Champion/0MARCAWG5VJK4GIDNLXR1CR0ON)